In [ ]:
# Licensing Information:  You are free to use or extend this project for
# educational purposes provided that (1) you do not distribute or publish
# solutions, (2) you retain this notice, and (3) you provide clear
# attribution to The Georgia Institute of Technology, including a link to https://aritter.github.io/CS-7650/

# Attribution Information: This assignment was developed at The Georgia Institute of Technology
# by Alan Ritter (alan.ritter@cc.gatech.edu)

# Project #2: Named Entity Recognition

In this assignment, you will implement a bidirectional LSTM-CNN-CRF for sequence labeling, following [this paper by Xuezhe Ma and Ed Hovy](https://www.aclweb.org/anthology/P16-1101.pdf), on the CoNLL named entity recognition dataset.  Before starting the assignment, we recommend reading the Ma and Hovy paper.

First, let's import some libraries and make sure the runtime has access to a GPU.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Wed Mar 17 17:57:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download the Data

Run the following code to download the English part of the CoNLL 2003 dataset, the evaluation script and pre-filtered GloVe embeddings we are providing for this data.

In [ ]:
#CoNLL 2003 data
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testb
!cat eng.train | awk '{print $1 "\t" $4}' > train
!cat eng.testa | awk '{print $1 "\t" $4}' > dev
!cat eng.testb | awk '{print $1 "\t" $4}' > test

#Evaluation Script
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl

#Pre-filtered GloVe embeddings
!wget https://raw.githubusercontent.com/aritter/aritter.github.io/master/files/glove.840B.300d.conll_filtered.txt

--2021-03-17 17:57:06--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3283420 (3.1M) [text/plain]
Saving to: ‘eng.train’

eng.train           100%[===================>]   3.13M  --.-KB/s    in 0.1s    

2021-03-17 17:57:07 (21.7 MB/s) - ‘eng.train’ saved [3283420/3283420]

--2021-03-17 17:57:07--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response.

## CoNLL Data Format

Run the following cell to see a sample of the data in CoNLL format.  As you can see, each line in the file represents a word and its labeled named entity tag in BIO format.  A blank line is used to seperate sentences.

In [ ]:
!head -n 20 train

-DOCSTART-	O
	
EU	I-ORG
rejects	O
German	I-MISC
call	O
to	O
boycott	O
British	I-MISC
lamb	O
.	O
	
Peter	I-PER
Blackburn	I-PER
	
BRUSSELS	I-LOC
1996-08-22	O
	
The	O
European	I-ORG


## Reading in the Data

Below we proivide a bit of code to read in data in the CoNLL format.  This also reads in the filtered GloVe embeddings, to save you some effort - we will discuss this more later.

In [ ]:
import re

#Read in the training data.
def read_conll_format(filename):
  (words, tags, currentSent, currentTags) = ([],[],['-START-'],['START'])
  for line in open(filename).readlines():
    line = line.strip()
    #print(line)
    if line == "":
      currentSent.append('-END-')
      currentTags.append('END')
      words.append(currentSent)
      tags.append(currentTags)
      (currentSent, currentTags) = (['-START-'], ['START'])
    else:
      (word, tag) = line.split()
      currentSent.append(word)
      currentTags.append(tag)
  return (words, tags)

#Read GloVe embeddings.
def read_GloVe(filename):
  embeddings = {}
  for line in open(filename).readlines():
    #print(line)
    fields = line.strip().split(" ")
    word = fields[0]
    embeddings[word] = [float(x) for x in fields[1:]]
  return embeddings

GloVe = read_GloVe("glove.840B.300d.conll_filtered.txt")

(sentences_train, tags_train) = read_conll_format("train")
(sentences_dev, tags_dev)     = read_conll_format("dev")

def sentences2char(sentences):
  return [[['start'] + [c for c in w] + ['end'] for w in l] for l in sentences]

sentencesChar = sentences2char(sentences_train)

print(sentences_train[2])
print(tags_train[2])

print(GloVe['the'])

['-START-', 'Peter', 'Blackburn', '-END-']
['START', 'I-PER', 'I-PER', 'END']
[0.27204, -0.06203, -0.1884, 0.023225, -0.018158, 0.0067192, -0.13877, 0.17708, 0.17709, 2.5882, -0.35179, -0.17312, 0.43285, -0.10708, 0.15006, -0.19982, -0.19093, 1.1871, -0.16207, -0.23538, 0.003664, -0.19156, -0.085662, 0.039199, -0.066449, -0.04209, -0.19122, 0.011679, -0.37138, 0.21886, 0.0011423, 0.4319, -0.14205, 0.38059, 0.30654, 0.020167, -0.18316, -0.0065186, -0.0080549, -0.12063, 0.027507, 0.29839, -0.22896, -0.22882, 0.14671, -0.076301, -0.1268, -0.0066651, -0.052795, 0.14258, 0.1561, 0.05551, -0.16149, 0.09629, -0.076533, -0.049971, -0.010195, -0.047641, -0.16679, -0.2394, 0.0050141, -0.049175, 0.013338, 0.41923, -0.10104, 0.015111, -0.077706, -0.13471, 0.119, 0.10802, 0.21061, -0.051904, 0.18527, 0.17856, 0.041293, -0.014385, -0.082567, -0.035483, -0.076173, -0.045367, 0.089281, 0.33672, -0.22099, -0.0067275, 0.23983, -0.23147, -0.88592, 0.091297, -0.012123, 0.013233, -0.25799, -0.02972, 0.0167

## Mapping Tokens to Indices

# As in the last project, we will need to convert words in the dataset to numeric indices, so they can be presented as input to a neural network.  Code to handle this for you with sample usage is provided below.

In [ ]:
#Create mappings between tokens and indices.

from collections import Counter
import random

#Will need this later to remove 50% of words that only appear once in the training data from the vocabulary (and don't have GloVe embeddings).
wordCounts = Counter([w for l in sentences_train for w in l])
charCounts = Counter([c for l in sentences_train for w in l for c in w])
singletons = set([w for (w,c) in wordCounts.items() if c == 1 and not w in GloVe.keys()])
charSingletons = set([w for (w,c) in charCounts.items() if c == 1])

#Build dictionaries to map from words, characters to indices and vice versa.
#Save first two words in the vocabulary for padding and "UNK" token.
word2i = {w:i+2 for i,w in enumerate(set([w for l in sentences_train for w in l] + list(GloVe.keys())))}
char2i = {w:i+2 for i,w in enumerate(set([c for l in sentencesChar for w in l for c in w]))}
i2word = {i:w for w,i in word2i.items()}
i2char  = {i:w for w,i in char2i.items()}

#Tag dictionaries.
tag2i = {w:i for i,w in enumerate(set([t for l in tags_train for t in l]))}
i2tag = {i:t for t,i in tag2i.items()}

#When training, randomly replace singletons with UNK tokens sometimes to simulate situation at test time.
def getDictionaryRandomUnk(w, dictionary, train=False):
  if train and (w in singletons and random.random() > 0.5):
    return 1
  else:
    return dictionary.get(w, 1)

#Map a list of sentences from words to indices.
def sentences2indices(words, dictionary, train=False):
  #1.0 => UNK
  return [[getDictionaryRandomUnk(w,dictionary, train=train) for w in l] for l in words]

#Map a list of sentences containing to indices (character indices)
def sentences2indicesChar(chars, dictionary):
  #1.0 => UNK
  return [[[dictionary.get(c,1) for c in w] for w in l] for l in chars]

#Indices
X       = sentences2indices(sentences_train, word2i, train=True)
X_char  = sentences2indicesChar(sentencesChar, char2i)
Y       = sentences2indices(tags_train, tag2i)

print(i2word[253])

#Print out some examples of what the dev inputs will look like
for i in range(10):
  print(" ".join([i2word.get(w,'UNK') for w in X[i]]))

Cito
-START- -DOCSTART- -END-
-START- EU rejects German call to boycott British lamb . -END-
-START- Peter Blackburn -END-
-START- BRUSSELS 1996-08-22 -END-
-START- The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep . -END-
-START- Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . -END-
-START- " We do n't support any such recommendation because we do n't see any grounds for it , " the Commission 's chief spokesman Nikolaus van der Pas told a news briefing . -END-
-START- He said further scientific study was required and if it was found that action was needed it should be taken by the European Union . -END-
-START- He said a proposal last month by EU Farm Commissioner Franz Fischler to ban sh

## Padding and Batching

In this assignment, you should train your models using minibatched SGD, rather than using a batch size of 1 as we did in the previous project.  When presenting multiple sentences to the network at the same time, we will need to pad them to be of the same length.

Below we provide some code to prepare batches of data to present to the network.  

**Side Note:** PyTorch includes utilities in [`torch.utils.data`](https://pytorch.org/docs/stable/data.html) to help with padding, batching, shuffling and some other things, but for this assignment we will do everything from scratch to help you see exactly how this works.

In [ ]:
#Pad inputs to max sequence length (for batching)
def prepare_input(X_list):
  X_padded = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(l) for l in X_list], batch_first=True).type(torch.LongTensor)
  X_mask   = torch.nn.utils.rnn.pad_sequence([torch.as_tensor([1.0] * len(l)) for l in X_list], batch_first=True).type(torch.FloatTensor)
  return (X_padded, X_mask)

#Maximum word length (for character representations)
MAX_CLEN=32

def prepare_input_char(X_list):
  MAX_SLEN = max([len(l) for l in X_list])
  X_padded  = [l + [[]]*(MAX_SLEN-len(l))  for l in X_list]
  X_padded  = [[w[0:MAX_CLEN] for w in l] for l in X_padded]
  X_padded  = [[w + [1]*(MAX_CLEN-len(w)) for w in l] for l in X_padded]
  #X_mask    = [[1]*len(l) + [0]*(MAX_SLEN-len(l)) for l in X_list]
  return torch.as_tensor(X_padded).type(torch.LongTensor)

#Pad outputs using one-hot encoding
def prepare_output_onehot(Y_list, NUM_TAGS=max(tag2i.values())+1):
  Y_onehot = [torch.zeros(len(l), NUM_TAGS) for l in Y_list]
  for i in range(len(Y_list)):
   for j in range(len(Y_list[i])):
    Y_onehot[i][j,Y_list[i][j]] = 1.0
  Y_padded = torch.nn.utils.rnn.pad_sequence(Y_onehot, batch_first=True).type(torch.FloatTensor)
  return Y_padded

print("max slen:", max([len(x) for x in X_char]))  #Max sequence length in the training data is 39.

(X_padded, X_mask) = prepare_input(X)
X_padded_char = prepare_input_char(X_char)
Y_onehot = prepare_output_onehot(Y)

print("X_padded:", X_padded.shape)
print("X_padded_char:", X_padded_char.shape)
print("Y_onehot:", Y_onehot.shape)

max slen: 115
X_padded: torch.Size([14987, 115])
X_padded_char: torch.Size([14987, 115, 32])
Y_onehot: torch.Size([14987, 115, 10])


## **Your code starts here:** Basic LSTM Tagger (10 points)

OK, now you should have everything you need to get started.

Recall that your goal is to to implement the BiLSTM-CNN-CRF, as described in [(Ma and Hovy, 2016)](https://www.aclweb.org/anthology/P16-1101.pdf).  This is a relatively complex network with various components.  Below we provide starter code to break down your implementation into increasingly complex versions of the final model, starting with a Basic LSTM tagger.  This way you can be confident that each part is working correctly before incrementally increasing the complexity of your implementation.  This is generally a good approach to take when implementing complex models, since buggy PyTorch code is often partially working, but produces worse results than a correct implementation, so it's hard to know whether added complexities are helping or hurting.  Also, if you aren't able to match published results it's hard to know which component of your model has the problem (or even whether or not it is a problem in the published result!)

Fill in the functions marked as `TODO` in the code block below.  If everything is working correctly, you should be able to achieve an F1 score of 0.87 on the dev set and 0.83 on the test set (with GloVe embeddings).

In [ ]:
class BasicLSTMtagger(nn.Module):
    def __init__(self, DIM_EMB=10, DIM_HID=10):
        super(BasicLSTMtagger, self).__init__()
        NUM_TAGS = max(tag2i.values())+1

        print(len(word2i))
        print(MAX_CLEN)
        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
        #TODO: initialize parameters - embedding layer, nn.LSTM, nn.Linear and nn.LogSoftmax
        #CHECK THIS
        # self.embed = nn.Embedding(len(word2i) + 2, DIM_EMB)
        self.init_glove(GloVe)
        self.drop = torch.nn.Dropout(p=0.5)
        self.lstm = nn.LSTM(DIM_EMB, DIM_HID, batch_first=True, dropout=0.5, bidirectional=True)
        self.fc = nn.Linear(DIM_HID*2, NUM_TAGS)
        self.softmax = nn.LogSoftmax(dim=2)
        # only arg to this is which dimension to compute the softmax along

    def forward(self, X, train=False):
        #TODO: Implement the forward computation.
        embeddings = self.embed(X)
        # print("embeddings.shape",embeddings.shape)
        x = self.drop(embeddings)
        out, hidden = self.lstm(x)
        tag_scores = self.fc(out)
        tag_probs = self.softmax(tag_scores)
        return tag_probs

    def init_glove(self, GloVe):
      #TODO: initialize word embeddings using GloVe (you can skip this part in your first version, if you want, see instructions below).
      glove_pretrain = torch.rand([len(word2i) + 2, self.DIM_EMB])
      for item in GloVe.items():
        glove_pretrain[word2i.get(item[0], 1), :] = torch.FloatTensor(item[1])

      self.embed = nn.Embedding.from_pretrained(glove_pretrain, freeze = False)
      

    def inference(self, sentences):
      X = prepare_input(sentences2indices(sentences, word2i))[0].cuda()
      pred = self.forward(X).argmax(dim=2)
      return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
      Y_pred = self.inference(words)
      for i in range(len(words)):
        print("----------------------------")
        print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
        print("Predicted:\t", Y_pred[i])
        print("Gold:\t\t", tags[i])

    def write_predictions(self, sentences, outFile):
      fOut = open(outFile, 'w')
      for s in sentences:
        y = self.inference([s])[0]
        #print("\n".join(y[1:len(y)-1]))
        fOut.write("\n".join(y[1:len(y)-1]))  #Skip start and end tokens
        fOut.write("\n\n")

#The following code will initialize a model and test that your forward computation runs without errors.
lstm        = BasicLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output = lstm.forward(prepare_input(X[0:5])[0])
Y_onehot    = prepare_output_onehot(Y[0:5])

#Check the shape of the lstm_output and one-hot label tensors.
print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

29146
32
lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
#Read in the data

(sentences_dev, tags_dev)     = read_conll_format('dev')
(sentences_train, tags_train) = read_conll_format('train')
(sentences_test, tags_test)   = read_conll_format('test')

# Train your Model (10 points)

Next, implement the function below to train your basic BiLSTM tagger.  See [torch.nn.lstm](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html).  Make sure to save your predictions on the test set (`test_pred_lstm.txt`) for submission to GradeScope.

In [ ]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random

def shuffle_sentences(sentences, tags):
  shuffled_sentences = []
  shuffled_tags      = []
  indices = list(range(len(sentences)))
  random.shuffle(indices)
  for i in indices:
    #print(len(sentences[i]), len(tags[i]))
    shuffled_sentences.append(sentences[i])
    shuffled_tags.append(tags[i])
  return (shuffled_sentences, shuffled_tags)

nEpochs = 10

def train_basic_lstm(sentences, tags, lstm):
  optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)

  batchSize = 10

  for epoch in range(nEpochs):
    totalLoss = 0.0

    (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
    for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):

      lstm.zero_grad()
      #TODO: Impelement gradient update.
      # inference
      batch_examples = sentences_shuffled[batch:min(len(sentences)-1, batch+batchSize)]
      batch_targets = tags_shuffled[batch:min(len(sentences)-1, batch+batchSize)]

      batch_examples_idx = prepare_input(sentences2indices(batch_examples, word2i))[0].cuda()
      batch_targets_idx = prepare_input(sentences2indices(batch_targets, tag2i))[0].cuda()
      batch_targets_onehot = prepare_output_onehot(batch_targets_idx).cuda()

      predictions = lstm.forward(batch_examples_idx).cuda()

      # compute loss
      loss = torch.sum(torch.einsum('bij,bij->bi', -predictions, batch_targets_onehot))

      totalLoss += loss
      # backprop and optimize
      loss.backward()
      optimizer.step()

    print(f"loss on epoch {epoch} = {totalLoss}")
    lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
    print('conlleval:')
    print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

    if epoch % 10 == 0:
      s = sample(range(len(sentences_dev)), 5)
      lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

lstm = BasicLSTMtagger(DIM_HID=500, DIM_EMB=300).cuda()
train_basic_lstm(sentences_train, tags_train, lstm)

29146
32


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


loss on epoch 0 = 53864.3359375
conlleval:
processed 51578 tokens with 5942 phrases; found: 5624 phrases; correct: 4203.
accuracy:  95.17%; precision:  74.73%; recall:  70.73%; FB1:  72.68
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  81.56%; recall:  78.99%; FB1:  80.25  1779
             MISC: precision:  65.25%; recall:  58.24%; FB1:  61.55  823
              ORG: precision:  61.80%; recall:  54.29%; FB1:  57.80  1178
              PER: precision:  80.68%; recall:  80.73%; FB1:  80.71  1843

----------------------------
-START-/START/START The/O/O titles/O/O of/O/O his/O/O other/O/O novels/O/O translate/O/O as/O/O "/O/O In/O/I-MISC the/O/I-MISC Year/I-MISC/I-MISC of/O/I-MISC January/O/I-MISC "/O/O (/O/O 1963/O/O )/O/O ,/O/O "/O/O The/O/I-MISC Collapse/O/I-MISC "/O/O (/O/O 1964/O/O )/O/O ,/O/O "/O/O Sleeping/O/I-MISC Bread/I-MISC/I-MISC "/O/O (/O/O 1975/O/O )/O/O ,/O/O "/O/O The/O/I-MISC Decaying/O/I-MISC Mansion/O/I-MISC "/O/O 

loss on epoch 1 = 26884.64453125
conlleval:
processed 51578 tokens with 5942 phrases; found: 5828 phrases; correct: 4653.
accuracy:  96.36%; precision:  79.84%; recall:  78.31%; FB1:  79.07
              LOC: precision:  84.45%; recall:  86.61%; FB1:  85.51  1884
             MISC: precision:  75.75%; recall:  60.30%; FB1:  67.15  734
              ORG: precision:  69.42%; recall:  64.65%; FB1:  66.95  1249
              PER: precision:  83.58%; recall:  88.98%; FB1:  86.20  1961



loss on epoch 2 = 20848.80859375
conlleval:
processed 51578 tokens with 5942 phrases; found: 5881 phrases; correct: 4868.
accuracy:  96.89%; precision:  82.78%; recall:  81.93%; FB1:  82.35
              LOC: precision:  85.92%; recall:  91.02%; FB1:  88.40  1946
             MISC: precision:  77.61%; recall:  65.40%; FB1:  70.98  777
              ORG: precision:  73.41%; recall:  71.44%; FB1:  72.41  1305
              PER: precision:  88.24%; recall:  88.76%; FB1:  88.50  1853



KeyboardInterrupt: ignored

In [ ]:
#Evaluation on test data
lstm.write_predictions(sentences_test, 'test_pred_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_lstm.txt | perl conlleval.pl -d "\t"

--2021-03-17 18:03:47--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.1’

conlleval.pl.1      100%[===================>]  12.46K  --.-KB/s    in 0s      

2021-03-17 18:03:47 (82.5 MB/s) - ‘conlleval.pl.1’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5612 phrases; correct: 4427.
accuracy:  96.09%; precision:  78.88%; recall:  78.38%; FB1:  78.63
              LOC: precision:  81.53%; recall:  85.49%; FB1:  83.47  1749
             MISC: precision:  67.77%; recall:  64.10%; FB1:  65.89  664
              ORG: precision:  71.67%; recall:  70.98%; FB1:  71.32  1645
              PER: precis

## Initialization with GloVe Embeddings (5 points)

If you haven't already, implement the `init_glove()` method in `BasicLSTMtagger` above.

Rather than initializing word embeddings randomly, it is common to use learned word embeddings (GloVe or Word2Vec), as discussed in lecture.  To make this simpler, we have already pre-filtered [GloVe](https://nlp.stanford.edu/projects/glove/) embeddings to only contain words in the vocabulary of the CoNLL NER dataset, and loaded them into a dictionary (`GloVe`) at the beginning of this notebook.



## Character Embeddings (10 points)

Now that you have your basic LSTM tagger working, the next step is to add a convolutional network that computes word embeddings from character representations of words.  See Figure 2 and Figure 3 in the [Ma and Hovy](https://www.aclweb.org/anthology/P16-1101.pdf) paper.  We have provided code in `sentences2input_tensors` to convert sentences into lists of word and character indices.  See also [nn.Conv1d](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html) and [MaxPool1d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html).

Make sure to save your predictions on the test set, for submission to GradeScope.  You should be able to achieve 90 F1 / 85 F1 on the dev/test sets.

In [ ]:
#For F.max_pool1d()
import torch.nn.functional as F

#class CharLSTMtagger(nn.Module):
class CharLSTMtagger(BasicLSTMtagger):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10):
        super(CharLSTMtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID)
        NUM_TAGS = max(tag2i.values())+1

        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
        #TODO: Initialize parameters.
        self.init_glove(GloVe)
        self.char_embed = nn.Embedding(len(char2i) + 2, DIM_CHAR_EMB)
        self.dropout = nn.Dropout(p=0.5)

        # Check this for padding later
        self.conv = nn.Conv1d(DIM_CHAR_EMB, DIM_CHAR_EMB, 3)

        # self.lstm = nn.LSTM(DIM_EMB + DIM_CHAR_EMB, DIM_HID, batch_first=True, dropout=0.5, bidirectional=True)
        self.lstm = nn.LSTM(DIM_EMB + DIM_CHAR_EMB, DIM_HID, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(DIM_HID*2, NUM_TAGS)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, X, X_char, train=False):
        #TODO: Implement the forward computation.
        # Get char embeddings
        char_embeddings = self.char_embed(X_char)

        # Dropout on Char embeddings
        x = self.dropout(char_embeddings)

        # CNN to get char representation
        x = x.permute(0, 1, 3, 2)
        x = torch.reshape(x,(x.shape[0] * x.shape[1], x.shape[2], x.shape[3]))
        x = self.conv(x)
        char_rep = F.max_pool1d(x, x.shape[2])
        char_rep = torch.reshape(char_rep,(char_embeddings.shape[0], 
                                           char_embeddings.shape[1],
                                           char_embeddings.shape[3]))

        # get word embeddings
        word_embeddings = self.embed(X)

        # Concat char representation with word embedding
        merged_representation = torch.cat((char_rep, word_embeddings), dim=2)

        # Droput on LSTM Input
        # x = self.dropout(merged_representation)
        # LSTM
        out, hidden = self.lstm(merged_representation)
        # # Dropout on LSTM Output
        # out = self.dropout(out)
        

        # Score to tag prob space
        tag_scores = self.fc(out)
        tag_probs = self.softmax(tag_scores)
        return tag_probs

    def sentences2input_tensors(self, sentences):
      (X, X_mask)   = prepare_input(sentences2indices(sentences, word2i))
      X_char        = prepare_input_char(sentences2indicesChar(sentences, char2i))
      return (X, X_mask, X_char)

    def inference(self, sentences):
      (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
      pred = self.forward(X.cuda(), X_char.cuda()).argmax(dim=2)
      return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
      Y_pred = self.inference(words)
      for i in range(len(words)):
        print("----------------------------")
        print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
        print("Predicted:\t", Y_pred[i])
        print("Gold:\t\t", tags[i])

lstm        = CharLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output = lstm.forward(prepare_input(X[0:5])[0], prepare_input_char(X_char[0:5]))
Y_onehot    = prepare_output_onehot(Y[0:5])

print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

29146
32


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


In [ ]:
#Training LSTM w/ character embeddings.

nEpochs = 10

def train_char_lstm(sentences, tags, lstm):
  optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)
  #TODO: initialize optimizer

  batchSize = 10

  for epoch in range(nEpochs):
    totalLoss = 0.0

    (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
    for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
      lstm.zero_grad()
      #TODO: Gradient update

      batch_examples = sentences_shuffled[batch:min(len(sentences)-1, batch+batchSize)]
      batch_targets = tags_shuffled[batch:min(len(sentences)-1, batch+batchSize)]

      X, X_mask, X_char = char_lstm.sentences2input_tensors(batch_examples)
      batch_targets_idx = prepare_input(sentences2indices(batch_targets, tag2i))[0].cuda()
      batch_targets_onehot = prepare_output_onehot(batch_targets_idx).cuda()

      predictions = char_lstm.forward(X.cuda(), X_char.cuda()).cuda()

      loss = torch.sum(torch.einsum('bij,bij->bi', -predictions, batch_targets_onehot))
    
      totalLoss += loss
      loss.backward()
      optimizer.step()


    print(f"loss on epoch {epoch} = {totalLoss}")
    lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
    print('conlleval:')
    print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

    if epoch % 10 == 0:
      s = sample(range(len(sentences_dev)), 5)
      lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

char_lstm = CharLSTMtagger(DIM_HID=500, DIM_EMB=300).cuda()
train_char_lstm(sentences_train, tags_train, char_lstm)

29146
32


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


KeyboardInterrupt: ignored

In [ ]:
#Evaluation on test set
char_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm.txt | perl conlleval.pl -d "\t"

--2021-03-17 18:05:50--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.2’

conlleval.pl.2      100%[===================>]  12.46K  --.-KB/s    in 0.001s  

2021-03-17 18:05:50 (17.7 MB/s) - ‘conlleval.pl.2’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 1818 phrases; correct: 726.
accuracy:  84.73%; precision:  39.93%; recall:  12.85%; FB1:  19.45
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  88.35%; recall:   5.46%; FB1:  10.28  103
             MISC: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              ORG: precision:  5

## Conditional Random Fields (15 points)

Now we are ready to add a CRF layer to the `CharacterLSTMTagger`.  To train the model, implement `conditional_log_likelihood`, using the score (unnormalized log probability) of the gold sequence, in addition to the partition function, $Z(X)$, which is computed using the forward algorithm.  Then, you can simply use Pytorch's automatic differentiation to compute gradients by running backpropagation through the computation graph of the dynamic program (this should be very simple, so long as you are able to correctly implement the forward algorithm using a computation graph that is supported by PyTorch).  This approach to computing gradients for CRFs is discussed in Section 7.5.3 of the [Eisenstein Book](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)

You will also need to implement the Viterbi algorithm for inference during decoding.

After including CRF training and Viterbi decoding, you should be getting about 92 F1 / 88 F1 on the dev and test set, respectively.

**IMPORTANT:** Note that training will be substantially slower this time - depending on the efficiency of your implementation, it could take about 5 minutes per epoch (e.g. 50 minutes for 10 iterations).  It is recommended to start out training on a single batch of data (and testing on this same batch), so that you can quickly debug, making sure your model can memorize the labels on a single batch, and then optimize your code.  Once you are fairly confident your code is working properly, then you can train using the full dataset.  We have provided a (commented out) line of code to switch between training on a single batch and the full dataset below.

**Hint #1:** While debugging your implementation of the Forward algorithm it is helpful to look at the loss during training.  The loss should never be less than zero (the log-likelihood should always be negative).

**Hint #2:** To sum log-probabilities in a numerically stable way at the end of the Forward algorithm, you will want to use [`torch.logsumexp`](https://pytorch.org/docs/stable/generated/torch.logsumexp.html).

In [ ]:
#For F.max_pool1d()
import torch.nn.functional as F

class LSTM_CRFtagger(CharLSTMtagger):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10, N_TAGS=max(tag2i.values())+1):
      super(LSTM_CRFtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID, DIM_CHAR_EMB=DIM_CHAR_EMB)

      #TODO: Initialize parameters.
      NUM_TAGS = max(tag2i.values())+1

      (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
      #TODO: Initialize parameters.
      self.init_glove(GloVe)
      self.char_embed = nn.Embedding(len(char2i) + 2, DIM_CHAR_EMB)
      self.dropout = nn.Dropout(p=0.5)

      # Check this for padding later
      self.conv = nn.Conv1d(DIM_CHAR_EMB, DIM_CHAR_EMB, 3)
      self.pool = nn.MaxPool1d(DIM_CHAR_EMB, 1)

      self.lstm = nn.LSTM(DIM_EMB + DIM_CHAR_EMB, DIM_HID, batch_first=True,
                          bidirectional=True)
      # self.lstm = nn.LSTM(DIM_EMB + DIM_CHAR_EMB, DIM_HID, batch_first=True,
      # dropout=0.5, bidirectional=True)

      self.fc = nn.Linear(DIM_HID*2, NUM_TAGS)

      self.transitionWeights = nn.Parameter(torch.zeros((N_TAGS, N_TAGS), requires_grad=True))
      nn.init.normal_(self.transitionWeights)

      self.START_IND = tag2i.get('START')
      self.END_IND = tag2i.get('END')

      self.transitionWeights.data[self.START_IND, :] = -10000.0
      self.transitionWeights.data[:, self.END_IND] = -10000.0


    def forward(self, X, X_char, train=False):
      #TODO: Implement the forward computation.
      # Get char embeddings
      char_embeddings = self.char_embed(X_char)

      # Dropout on Char embeddings
      x = self.dropout(char_embeddings)

      # CNN to get char representation
      x = x.permute(0, 1, 3, 2)
      x = torch.reshape(x,(x.shape[0] * x.shape[1], x.shape[2], x.shape[3]))
      x = self.conv(x)
      char_rep = self.pool(x)
      char_rep = torch.reshape(char_rep,(char_embeddings.shape[0], 
                                          char_embeddings.shape[1],
                                          char_embeddings.shape[3]))

      # get word embeddings
      word_embeddings = self.embed(X)

      # Concat char representation with word embedding
      merged_representation = torch.cat((char_rep, word_embeddings), dim=2)

      # Droput on LSTM Input
      x = self.dropout(merged_representation)
      # LSTM
      out, hidden = self.lstm(x)
      # Dropout on LSTM Output
      out = self.dropout(out)

      #After you take the dot product between the LSTM features and a 
      #weight vector for a particular label, this will give you the emission 
      #score for that input/label.
      tag_scores = self.fc(out)

      # Output should be of shape batchsize, max sentence length in words, Num tags(onehot vector)
      return tag_scores
      # these scores are then used below as input to the viterbi algo


### INFERENCE START

    # finds highest prob tag sequence for given input sentence
    # For inference only
    def viterbi(self, lstm_scores, sLen):
      #TODO: Implement Viterbi algorithm, soring backpointers to recover the argmax sequence.  Returns the argmax sequence in addition to its unnormalized conditional log-likelihood.
      
      backpointers = []
      # current_vs = lstm_scores[0].view(1, -1).cuda()
      
      # TESTING LINE
      # current_vs = torch.full((1, self.NUM_TAGS), -10000.0)
      current_vs = torch.zeros((1, self.NUM_TAGS)).cuda()
      current_vs[0,self.START_IND] = lstm_scores[0,self.START_IND]
      current_vs += self.transitionWeights[:, self.START_IND]

      # print("self.START_IND ", self.START_IND)
      # print("self.END_IND ", self.END_IND)
      # print("self.NUM_TAGS", self.NUM_TAGS)

      for word_ind in range(1, sLen - 1):
        backpointers_t = []  # holds the backpointers for this step
        next_vs = torch.zeros(self.NUM_TAGS).cuda()  # holds the viterbi variables for this step

        # Zeroing out start and end scores
        current_vs[0,self.START_IND] = 0.0
        current_vs[0,self.END_IND] = 0.0


        for next_tag in range(self.NUM_TAGS):
          # find v_t+1 + phi_trans(s_t+1|s_t) and assign backpointers
          tag_next_vs = current_vs.cuda() + self.transitionWeights[next_tag]
          # print("tag_next_vs.shape",next_tag_var.shape)
          best_tag_id = torch.argmax((tag_next_vs),1)
          # print("best_tag_id" ,best_tag_id)
          backpointers_t.append(best_tag_id)
          next_vs[next_tag] = tag_next_vs[0][best_tag_id].view(1)

        # phi_emission(x_t+1|s_t+1)
        word_scores = lstm_scores[word_ind]
        # Zeroing out start and end emission probs
        word_scores[self.START_IND] = 0.0
        word_scores[self.END_IND] = 0.0
        # v_t+1 = phi_trans(s_t+1 | s_t) + phi_emission(x_t+1|s_t+1)
        current_vs = (next_vs + word_scores).view(1, -1)
        backpointers.append(backpointers_t)

      current_vs += self.transitionWeights[self.END_IND]
      best_tag_id = torch.argmax((current_vs),1)

      # print("best_tag_id ", best_tag_id)
      # print("self.END_IND ", self.END_IND)
      path_score = current_vs[0][best_tag_id]

      # Follow the back pointers to decode the best path.
      best_path = [int(self.END_IND), int(best_tag_id)]
      for backpointers_t in reversed(backpointers):
        best_tag_id = backpointers_t[best_tag_id]
        best_path.append(int(best_tag_id))

      best_path.append(int(self.START_IND))
      best_path.reverse()
      
      # print(i2tag)
      # print("best_path", best_path)
      # for item in best_path:
      #   print(item, " is tag ",i2tag[item])
      
      return torch.tensor(best_path), path_score


    #Computes Viterbi sequences on a batch of data.
    def viterbi_batch(self, sentences):
      viterbiSeqs = []
      (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
      lstm_scores = self.forward(X.cuda(), X_char.cuda())
      # print("len(sentences)",len(sentences))
      for s in range(len(sentences)):
        # print("sentence", sentences[s])
        (viterbiSeq, ll) = self.viterbi(lstm_scores[s].cuda(), len(sentences[s]))
        viterbiSeqs.append(viterbiSeq)
      # print("viterbiSeqs",viterbiSeqs)
      return viterbiSeqs

### INFERENCE END

### TRAINING START

    #Forward algorithm for a single sentence
    #Efficiency will eventually be important here.  We recommend you start by 
    #training on a single batch and make sure your code can memorize the 
    #training data.  Then you can go back and re-write the inner loop using 
    #tensor operations to speed things up.

    # computes paritition function P(Z) or the denominator of the CRF
    def forward_algorithm(self, lstm_scores, sLen):
      #TODO: implement forward algorithm.
      current_alphas = torch.zeros((1, self.NUM_TAGS)).cuda()
      current_alphas[0,self.START_IND] = lstm_scores[0,self.START_IND]
      current_alphas += self.transitionWeights[:, self.START_IND]

      # Iterate through the sentence
      for word_ind in range(1, sLen - 1):
        next_alphas = torch.zeros(self.NUM_TAGS).cuda()
        word_score = lstm_scores[word_ind]

        # Zeroing out start and end scores  
        current_alphas[0,self.START_IND] = 0.0
        current_alphas[0,self.END_IND] = 0.0
        
        # Zeroing out start and end emission probs
        word_score[self.START_IND] = 0.0
        word_score[self.END_IND] = 0.0

        for next_tag in range(self.NUM_TAGS):
          # computing a_t+1 = sum a_t(s_t) + phi_trans(s_t+1 | s_t) + phi_emission(x_t+1|s_t+1)
          emission_potential = word_score[next_tag].view(1, -1).expand(1, self.NUM_TAGS).cuda()
          transmission_potential = self.transitionWeights[next_tag].view(1, -1).cuda()
          tag_next_alpha = (current_alphas + transmission_potential + emission_potential).cuda()
          next_alphas[next_tag] = torch.logsumexp(tag_next_alpha,dim=1).view(1).cuda()

        current_alphas = next_alphas.view(1, -1)
      current_alphas += self.transitionWeights[self.END_IND] 
      return torch.logsumexp(current_alphas,dim=1)
      # returning log of partition function

    def gold_score(self, lstm_scores, Y):
      #TODO: compute score of gold sequence Y (unnormalized conditional log-probability)
      score = torch.clone(lstm_scores[0,Y[0]]).cuda()
      
      for i in range(Y.shape[0] - 1):
          if (Y[i] == self.END_IND):
            break
          word_score = lstm_scores[i].cuda()
          score = score + self.transitionWeights[Y[i + 1], Y[i]] + word_score[Y[i + 1]]
      return score


    def conditional_log_likelihood(self, sentences, tags, train=True):
      #Todo: compute conditional log likelihood of Y (use forward_algorithm and gold_score)

      (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
      lstm_scores = self.forward(X.cuda(), X_char.cuda()).cuda()

      tags_idx = prepare_input(sentences2indices(tags, tag2i))[0].cuda()

      cond_log_likelihoods = torch.zeros([len(sentences)]) # size of batch so num of sentences per batch

      for s in range(len(sentences)):
        # print("sentence ", s)
        # print("sentences[s] ", sentences[s])
        forward_score = self.forward_algorithm(lstm_scores[s], len(sentences[s])).cuda() # May need to use something other than len here
        gold_score = self.gold_score(lstm_scores[s].cuda(), tags_idx[s].cuda())

        # print("forward_score: ", forward_score)
        # print("gold_score: ", gold_score)

        # THIS IS WHAT WAS ON PIAZZA
        cond_log_likelihoods[s] = forward_score - gold_score
        # print("cond_log_likelihoods[s]: ", s, cond_log_likelihoods[s])
      
      loss = torch.sum(cond_log_likelihoods)
      # print("cond_log_likelihood batch", loss, "\n\n")
      return loss


### TRAINING END


    def print_predictions(self, words, tags):
      Y_pred = self.inference(words)
      for i in range(len(words)):
        print("----------------------------")
        print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
        print("Predicted:\t", [Y_pred[i][j] for j in range(len(words[i]))])
        print("Gold:\t\t", tags[i])

    #Need to use Viterbi this time.
    # REVERT THIS LATER
    def inference(self, sentences, viterbi=True):
      pred = self.viterbi_batch(sentences)
      return [[i2tag[pred[i][j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]
  
lstm_crf = LSTM_CRFtagger(DIM_EMB=300).cuda()   

29146
32


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
print(lstm_crf.conditional_log_likelihood(sentences_dev[0:3], tags_dev[0:3]))

tensor(33.9177, grad_fn=<SumBackward0>)


In [ ]:
#CharLSTM-CRF Training
import tqdm
import os
import subprocess
import random

nEpochs=10

#Get CoNLL evaluation script
os.system('wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl')

def train_crf_lstm(sentences, tags, lstm):
  optimizer = optim.Adadelta(lstm.parameters(), lr=1.0)
  #TODO: initialize optimizer

  batchSize = 10

  for epoch in range(nEpochs):
    totalLoss = 0.0
    lstm.train()

    #Shuffle the sentences
    (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
    for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
      lstm.zero_grad()
      #TODO: take gradient step on a batch of data.

      batch_examples = sentences_shuffled[batch:min(len(sentences)-1, batch+batchSize)]
      batch_targets = tags_shuffled[batch:min(len(sentences)-1, batch+batchSize)]

      loss = lstm.conditional_log_likelihood(batch_examples, batch_targets)
      # print("loss", loss)
    
      totalLoss += loss
      loss.backward()
      optimizer.step()


    print(f"loss on epoch {epoch} = {totalLoss}")
    lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
    print('conlleval:')
    print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

    if epoch % 10 == 0:
      lstm.eval()
      s = random.sample(range(50), 5)
      lstm.print_predictions([sentences_train[i] for i in s], [tags_train[i] for i in s])   #Print predictions on train data (useful for debugging)

crf_lstm = LSTM_CRFtagger(DIM_HID=500, DIM_EMB=300, DIM_CHAR_EMB=30).cuda()

#REMEMBER TO CHANGE THIS BACK

# I just cant any more I literally lost my mind trying to debug this function.
# Im just a fucking mess and I cant fix this issue after days and days of trying
# and I have to catch up on my other classes

train_crf_lstm(sentences_train, tags_train, crf_lstm)             #Train on the full dataset
# train_crf_lstm(sentences_train[0:50], tags_train[0:50], crf_lstm)          #Train only the first batch (use this during development/debugging)

29146
32


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
crf_lstm.eval()
crf_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm_crf.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm_crf.txt | perl conlleval.pl -d "\t"

NameError: ignored

## Gradescope

Gradescope allows you to add multiple files to your submission. Please submit this notebook along with the test set prediction:
* test_pred_lstm.txt
* test_pred_cnn_lstm.txt
* test_pred_cnn_lstm_crf.txt
* NER_release.ipynb

To download this notebook, go to `File > Download.ipynb`. You can download the predictions from Colab by clicking the folder icon on the left and finding them under Files. 

Please make sure that you name the files as specified above. You will be able to see the test set accuracy for your predictions. However, the final score will be assigned later based on accuracy and implementation. 

When submitting the .ipynb notebook, please make sure that all the cells run when executed in order starting from a fresh session. If the code doesn't take too long to run, you can re-run everything with `Runtime -> Restart and run all`

You can submit multiple times before the deadline and choose the submission which you want to be graded by going to `Submission History` on gradescope.
